In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
# get teamfights winstons lab data for this season -
# query made thorugh gui and copy the link manually
# TODO - auto generate query

url = "https://www.winstonslab.com/customquery/completeFightInfo/?dateGreater=&dateSmaller=&event%5B%5D=130&specificMatchupTeam1=0&specificMatchupTeam2=0&team%5B%5D=&map%5B%5D=&roundtype%5B%5D=&fightlength=0&killsmade=1&ultsused=0&heroes=&heroesUlted="

In [3]:
# requests does not wait for the query to load in browser...
# use selenium instead to open the page and wait for it to load
# in python3 maybe can use urllib instead of requests?

driver = webdriver.Chrome()
driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, "lxml")

In [4]:
html_table = soup.find("table")
# need string object for pd parsing
str_html_table = str(html_table)

In [19]:
df = pd.read_html(str_html_table, displayed_only=False)[0]

In [20]:
df.head()

,Map,Roundtype,Blue Team,Red Team,time?,len?,KB?,KR?,UB?,UR?,FB?,Death Sequence?,Ults Blue,Ults Red,Teamcomp Blue,Teamcomp Red
0,Ilios,Lighthouse,PHI,LDN,0:59,16,0,6,0,0,R,NaN,NaN,NaN,NaN,NaN
1,Ilios,Lighthouse,PHI,LDN,1:48,36,3,5,6,6,B,NaN,NaN,NaN,NaN,NaN
2,Ilios,Lighthouse,PHI,LDN,2:53,11,0,6,0,0,R,NaN,NaN,NaN,NaN,NaN
3,Ilios,Lighthouse,PHI,LDN,3:27,10,0,6,1,2,R,NaN,NaN,NaN,NaN,NaN
4,Ilios,Ruins,PHI,LDN,0:29,0,1,0,0,0,B,NaN,NaN,NaN,NaN,NaN


In [21]:
# unfortunately does not pick up on the ults and death sequence - creating manual parsing to extract

img_rows = []
for tr in html_table.find_all("tr"):
    row = []
    tds = tr.find_all("td")
    if len(tds) == 0:
        continue
    # death sequence
    death_seq = []
    for img in tds[11]:
        death_seq.append({"team": img['class'][0], "hero": img['title']})
    row.append(death_seq)
    # ults
    for img_list in tds[12:]:
        row.append([img['title'] for img in img_list])
    img_rows.append(row)

# replace in df
img_cols = list(df.columns[-5:])
df[img_cols] = img_rows

In [30]:
# view df to check work
df.head()

,Map,Roundtype,Blue Team,Red Team,time?,len?,KB?,KR?,UB?,UR?,FB?,Death Sequence?,Ults Blue,Ults Red,Teamcomp Blue,Teamcomp Red
0,Ilios,Lighthouse,PHI,LDN,0:59,16,0,6,0,0,R,"[{u'hero': u'Zenyatta', u'team': u'blue'}, {u'...",[],[],"[Brigitte, Lucio, Zenyatta, D.Va, Reinhardt, Z...","[Brigitte, Lucio, Zenyatta, D.Va, Winston, Zarya]"
1,Ilios,Lighthouse,PHI,LDN,1:48,36,3,5,6,6,B,"[{u'hero': u'Winston', u'team': u'red'}, {u'he...","[Brigitte, Zenyatta, Lucio, Reinhardt, Zarya, ...","[Zarya, Winston, Brigitte, D.Va, Zenyatta, Lucio]","[Brigitte, Lucio, Zenyatta, D.Va, Reinhardt, Z...","[Brigitte, Lucio, Zenyatta, D.Va, Winston, Zarya]"
2,Ilios,Lighthouse,PHI,LDN,2:53,11,0,6,0,0,R,"[{u'hero': u'Reinhardt', u'team': u'blue'}, {u...",[],[],"[Brigitte, Lucio, Zenyatta, D.Va, Reinhardt, Z...","[Brigitte, Lucio, Zenyatta, D.Va, Reinhardt, Z..."
3,Ilios,Lighthouse,PHI,LDN,3:27,10,0,6,1,2,R,"[{u'hero': u'Zarya', u'team': u'blue'}, {u'her...",[Brigitte],"[Reinhardt, Brigitte]","[Brigitte, Lucio, Zenyatta, D.Va, Reinhardt, Z...","[Brigitte, Lucio, Zenyatta, D.Va, Reinhardt, Z..."
4,Ilios,Ruins,PHI,LDN,0:29,0,1,0,0,0,B,"[{u'hero': u'Zenyatta', u'team': u'red'}]",[],[],"[Brigitte, Lucio, Zenyatta, D.Va, Winston, Zarya]","[Brigitte, Lucio, Zenyatta, D.Va, Winston, Zarya]"


In [46]:
# we have ~2k team fights thus far in the season
df.shape

(2165, 21)

In [31]:
# create winning team column
# assume winning team has the most kills total in the fight 
# so you can win the team fight but still lose the point from c9 for example

def count_helper(death_seq, color):
    deaths = 0
    for el in death_seq:
        if el['team'] == color:
            deaths += 1
    return deaths
            
df['death_red'] = df['Death Sequence?'].apply(lambda x: count_helper(x, 'red'))
df['death_blue'] = df['Death Sequence?'].apply(lambda x: count_helper(x, 'blue'))
df['winner'] = df.apply(lambda x: 'red' if x['death_red'] < x['death_blue'] 
                        else ('tie' if x['death_red'] == x['death_blue'] else'blue'), 
                        axis=1)

In [32]:
# break out first pick for ease
df['first_pick_hero'] = df['Death Sequence?'].apply(lambda x: x[0]['hero'])
df['first_pick_team'] = df['Death Sequence?'].apply(lambda x: x[0]['team'])

In [95]:
# describe fights where winner is red or blue
df[df['winner'] == 'red'].describe()

,len?,KB?,KR?,UB?,UR?,death_red,death_blue
count,1042.000000,1042.000000,1042.000000,1042.000000,1042.000000,1042.000000,1042.000000
mean,16.002879,0.553743,4.121881,1.281190,1.803263,0.553743,4.121881
std,11.768704,0.922288,2.062315,1.476883,1.660026,0.922288,2.062315
min,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,8.000000,0.000000,2.000000,0.000000,0.000000,0.000000,2.000000
50%,14.000000,0.000000,4.000000,1.000000,2.000000,0.000000,4.000000
75%,22.000000,1.000000,6.000000,2.000000,3.000000,1.000000,6.000000
max,76.000000,6.000000,13.000000,8.000000,8.000000,6.000000,13.000000


In [96]:
# seems to have similar statistics for red/blue - teamfights on attacking/defending seems balanced
df[df['winner'] == 'blue'].describe()

,len?,KB?,KR?,UB?,UR?,death_red,death_blue
count,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000
mean,16.300289,4.298364,0.547642,1.878730,1.405197,4.298364,0.547642
std,12.561690,2.173475,0.932909,1.749407,1.579740,2.173475,0.932909
min,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,8.000000,3.000000,0.000000,0.000000,0.000000,3.000000,0.000000
50%,15.000000,5.000000,0.000000,2.000000,1.000000,5.000000,0.000000
75%,22.000000,6.000000,1.000000,3.000000,2.000000,6.000000,1.000000
max,141.000000,17.000000,6.000000,11.000000,8.000000,17.000000,6.000000


In [117]:
# first pick win percentage
# if you get the first pick, you have an 88% chance of winning the teamfight
df[df['first_pick_team'] != df['winner']].shape[0]/float(df.shape[0])

0.8859122401847576

In [35]:
# get perecentage of time the hero is the first to die in a teamfight 
# also calculate after that death the percentage of time the opposing team wins the teamfight

first_pick_rates = (df['first_pick_hero'].value_counts()/float(df.shape[0])).to_frame()

win_percentage_after_pick = []

for hero in df['first_pick_hero'].unique():
    hero_df = df[df['first_pick_hero'] == hero]
    win_percentage = hero_df[hero_df['first_pick_team'] != hero_df['winner']].shape[0]/float(hero_df.shape[0])
    win_percentage_after_pick.append([hero, win_percentage])

In [40]:
# merge dfs
first_pick_rates = first_pick_rates.reset_index().rename(columns={"index": "hero", "first_pick_hero": "perc_picked_first"})
first_pick_percentage_df = pd.DataFrame(win_percentage_after_pick, columns=['hero', 'win_percentage_after_pick'])
first_pick_percentage_df = first_pick_percentage_df.merge(first_pick_rates, how='left', on='hero')

In [47]:
# view results
first_pick_percentage_df.sort_values(by="perc_picked_first", ascending=False)

,hero,win_percentage_after_pick,perc_picked_first
2,Reinhardt,0.913333,0.207852
5,Brigitte,0.869081,0.165820
0,Zenyatta,0.903226,0.128868
6,Lucio,0.878327,0.121478
3,Zarya,0.914980,0.114088
1,Winston,0.859155,0.065589
7,D.Va,0.912500,0.036952
12,Sombra,0.913043,0.031871
13,Hammond,0.854839,0.028637
15,Ana,0.800000,0.020785
